In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np
import matplotlib.pyplot as plt
import cv2
from PIL import ImageGrab
from lib.SQL import SQLCalls
from keras.preprocessing.sequence import pad_sequences
try:
    xrange = xrange
except:
    xrange = range

Using TensorFlow backend.


In [2]:
def discount_rewards(r):
    """ take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(xrange(0, r.size)):
        running_add = running_add * .99 + r[t]
        discounted_r[t] = running_add   
    return discounted_r

def process_img(original_image):
    processed_img=cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)
    processed_img= cv2.resize(processed_img,(28,28))
    return processed_img

In [3]:
def length(sequence):
    used = tf.sign(tf.reduce_max(tf.abs(sequence), reduction_indices=2))
    length = tf.reduce_sum(used, reduction_indices=1)
    length = tf.cast(length, tf.int32)
    return length

In [17]:
def run():
    tf.reset_default_graph()
    myAgent = agent(lr=1e-2,s_size=28*28,a_size=4,h_size=64)
    init = tf.global_variables_initializer()
    update_frequency = 5
    init = tf.global_variables_initializer()
    d=False
    SQL=SQLCalls()
    Genomes=SQL.GatherGenomes()
    # Launch the tensorflow graph
    with tf.Session() as sess:
        sess.run(init)
        i = 0
        total_reward = []
        total_lenght = []
        UsedGenomes=np.ones(24)
        gradBuffer = sess.run(tf.trainable_variables())
        for ix,grad in enumerate(gradBuffer):
            gradBuffer[ix] = grad * 0

        while i < 1:
            s= print_screen = np.array(ImageGrab.grab(bbox=(0,60,580,530)))
            s=np.array(np.reshape(process_img(print_screen),[-1]))
            running_reward = 0
            ep_history = []
            for j in range(1):
                #Probabilistically pick an action given our network outputs.
                a_dist = sess.run(myAgent.output,feed_dict={myAgent.state_in:[s],myAgent.genomes:Genomes\
                                                             ,myAgent.used_genomes:UsedGenomes})
                a = np.random.choice(a_dist,p=a_dist)
                a = np.argmax(a_dist == a)
                print(a)
                #running_reward += r
                if d == True:
                    #Update the network.
                    ep_history = np.array(ep_history)
                    ep_history[:,2] = discount_rewards(ep_history[:,2])
                    feed_dict={myAgent.reward_holder:ep_history[:,2],
                            myAgent.action_holder:ep_history[:,1],myAgent.state_in:np.vstack(ep_history[:,0])}
                    grads = sess.run(myAgent.gradients, feed_dict=feed_dict)
                    for idx,grad in enumerate(grads):
                        gradBuffer[idx] += grad

                    if i % update_frequency == 0 and i != 0:
                        feed_dict= dictionary = dict(zip(myAgent.gradient_holders, gradBuffer))
                        _ = sess.run(myAgent.update_batch, feed_dict=feed_dict)
                        for ix,grad in enumerate(gradBuffer):
                            gradBuffer[ix] = grad * 0

                    total_reward.append(running_reward)
                    total_lenght.append(j)
                    break


            i += 1

In [18]:
#https://gist.github.com/danijar/3f3b547ff68effb03e20c470af22c696
#https://danijar.com/variable-sequence-lengths-in-tensorflow/
#https://stackoverflow.com/questions/36817596/get-last-output-of-dynamic-rnn-in-tensorflow
def length(sequence):
    used = tf.sign(tf.reduce_max(tf.abs(sequence), reduction_indices=2))
    length = tf.reduce_sum(used, reduction_indices=1)
    length = tf.cast(length, tf.int32)
    return length

def last_relevant(output, length):
    batch_size = tf.shape(output)[0]
    max_length = int(output.get_shape()[1])
    output_size = int(output.get_shape()[2])
    index = tf.range(0, batch_size) * max_length + (length - 1)
    flat = tf.reshape(output, [-1, output_size])
    relevant = tf.gather(flat, index)
    return relevant

class agent():
    def __init__(self, lr, s_size,a_size,h_size):
        #These lines established the feed-forward part of the network. The agent takes a state and produces an action.
        self.state_in= tf.placeholder(shape=[None,s_size],dtype=tf.float32)
        self.genomes= tf.placeholder(shape=[24,8,3],dtype=tf.int32)
        self.used_genomes= tf.placeholder(shape=[24],dtype=tf.int32)
        #self.genomes=tf.placeholder(tf.float32, [None, max_length, frame_size])
        
        
        hidden = slim.fully_connected(self.state_in,h_size,biases_initializer=None,activation_fn=tf.nn.relu)
        sequence_output, state = tf.nn.dynamic_rnn(tf.contrib.rnn.LSTMCell(64),\
                                          tf.cast(self.genomes,tf.float32),dtype=tf.float32,sequence_length=length(self.genomes))
        last = last_relevant(sequence_output, length(self.genomes))
        
        self.hidden2= slim.fully_connected(last,1,biases_initializer=None,activation_fn=tf.nn.sigmoid)
        self.output=tf.reshape(self.hidden2,[-1])*tf.cast(self.used_genomes,tf.float32)
        self.output=tf.nn.softmax(self.output)
        #self.chosen_action = tf.argmax(self.output,1)


               # hidden3= tf.concat(tf.to_float(hidden),tf.to_float(hidden2))
        #The next six lines establish the training proceedure. We feed the reward and chosen action into the network
        #to compute the loss, and use it to update the network.
#         self.reward_holder = tf.placeholder(shape=[None],dtype=tf.float32)
#         self.action_holder = tf.placeholder(shape=[None],dtype=tf.int32)
        
#         self.indexes = tf.range(0, tf.shape(self.output)[0]) * 2 + self.action_holder
#         self.responsible_outputs = tf.gather(tf.reshape(self.output, [-1]), self.indexes)
#         self.loss = -tf.reduce_mean(tf.log(self.responsible_outputs)*self.reward_holder)
        
#         tvars = tf.trainable_variables()
#         self.gradient_holders = []
#         for idx,var in enumerate(tvars):
#             placeholder = tf.placeholder(tf.float32,name=str(idx)+'_holder')
#             self.gradient_holders.append(placeholder)
        
#         self.gradients = tf.gradients(self.loss,tvars)
        
#         optimizer = tf.train.AdamOptimizer(learning_rate=lr)
#         self.update_batch = optimizer.apply_gradients(zip(self.gradient_holders,tvars))
run()

(24, 1)
10


In [7]:
x=[[ .5,.5]]
a = np.random.choice(x[0],p=x[0])
a = np.argmax(x == a)